In [2]:
import os
import math
import csv
import collections
import pdb

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor

%matplotlib inline

In [3]:
#path_training_dataset = "training_dataset.csv"
df_data = pd.read_csv('training_data.csv')

C:\Users\vishn\AppData\Local\Temp\ipykernel_29528\2851635126.py:2: DtypeWarning: Columns (6,54) have mixed types. Specify dtype option on import or set low_memory=False.
  df_data = pd.read_csv('training_data.csv')


In [4]:
#Top records
df_data.head()

,PolicyNo,Policy_Company,Policy_Installment_Term,Policy_Billing_Code,Policy_Method_Of_Payment,Policy_Reinstatement_Fee_Indicator,Policy_Zip_Code_Garaging_Location,Vehicle_Territory,Vehicle_Make_Year,Vehicle_Make_Description,...,EEA_Prior_Bodily_Injury_Limit,EEA_PolicyYear,SYS_Renewed,SYS_New_Business,Annual_Premium,Claim_Count,Loss_Amount,Frequency,Severity,Loss_Ratio
0,164532941,Standard,6,Direct Billed to Insured,Pre-paid,N,43046,35,2004,BUIK LESABRE LI,...,100-400,2006,Y,N,320.12,0,0.0,0.0,0.0,0.0
1,164533241,Standard,6,Direct Billed to Insured,Pre-paid,N,Unknown,35,1980,CADILLAC 4-DOOR,...,100-200,2006,Y,N,259.70,0,0.0,0.0,0.0,0.0
2,164534633,Standard,6,Direct Billed to Insured,Pre-paid,N,43555,17,2005,PONT MONTANA SV,...,100-400,2006,Y,N,613.74,0,0.0,0.0,0.0,0.0
3,164534839,Standard,6,Direct Billed to Insured,Pre-paid,N,43561,17,2005,MERC GRAND MARQ,...,40-100,2006,Y,N,541.66,0,0.0,0.0,0.0,0.0
4,164534840,Standard,6,Direct Billed to Insured,Pre-paid,N,43561,17,2005,MERC GRAND MARQ,...,40-100,2006,Y,N,541.66,0,0.0,0.0,0.0,0.0


In [5]:
# Data type of all columns

df_data.columns

Index(['PolicyNo', 'Policy_Company', 'Policy_Installment_Term',
       'Policy_Billing_Code', 'Policy_Method_Of_Payment',
       'Policy_Reinstatement_Fee_Indicator',
       'Policy_Zip_Code_Garaging_Location', 'Vehicle_Territory',
       'Vehicle_Make_Year', 'Vehicle_Make_Description', 'Vehicle_Performance',
       'Vehicle_New_Cost_Amount', 'Vehicle_Symbol',
       'Vehicle_Number_Of_Drivers_Assigned', 'Vehicle_Usage',
       'Vehicle_Miles_To_Work', 'Vehicle_Days_Per_Week_Driven',
       'Vehicle_Annual_Miles', 'Vehicle_Anti_Theft_Device',
       'Vehicle_Passive_Restraint', 'Vehicle_Age_In_Years',
       'Vehicle_Med_Pay_Limit', 'Vehicle_Bodily_Injury_Limit',
       'Vehicle_Physical_Damage_Limit',
       'Vehicle_Comprehensive_Coverage_Indicator',
       'Vehicle_Comprehensive_Coverage_Limit',
       'Vehicle_Collision_Coverage_Indicator',
       'Vehicle_Collision_Coverage_Deductible', 'Driver_Total',
       'Driver_Total_Male', 'Driver_Total_Female', 'Driver_Total_Single',
     

In [6]:
#understanding the numeric columns in the data frame value distribution
#df.describe()

In [7]:
# Common Functions

In [8]:
def get_selected_features(feature_list_file_path):
    with open(feature_list_file_path) as f:
        features_list = f.read().splitlines() 
        return features_list

In [9]:
def strip_white_spaces(input_df):
    columns = list(input_df.columns)
    for column in columns:
        input_df[column] = input_df[column].str.strip()
    return input_df

In [10]:
def convert_df_objects_to_str(df_dataset, df_columns):
    for column in df_columns:
        if df_dataset[column].dtype == 'object':
            df_dataset[column] = df_dataset[column].astype(str)
    return df_dataset

In [11]:
def get_num_cat_features(df_dataset, df_columns):
    categorical_variables = []
    numerical_variables = []
    for column in df_columns:
        print(column)
        if df_dataset[column].dtype == 'object':
            categorical_variables.append(column)
        else:
            numerical_variables.append(column)
    return numerical_variables, categorical_variables

In [12]:
selected_features_train = [
    'Vehicle_Youthful_Good_Student_Code',
    'Vehicle_Driver_Points',
    'Driver_Total_Female',
    'Driver_Total_Male',
    'Driver_Total_Teenager_Age_15_19',
    'Driver_Total_College_Ages_20_23',
    'Driver_Total_Young_Adult_Ages_24_29',
    'Driver_Total_Low_Middle_Adult_Ages_30_39',
    'Driver_Total_Middle_Adult_Ages_40_49',
    'Driver_Total_Adult_Ages_50_64',
    'Driver_Total_Senior_Ages_65_69',
    'Driver_Total_Upper_Senior_Ages_70_plus',
    'Driver_Total_Married',
    'Driver_Total_Single',
    'Vehicle_Usage',
    'Vehicle_Miles_To_Work',
    'Vehicle_Territory',
    'Annual_Premium',
    'Loss_Amount']

In [13]:
df_data = df_data[selected_features_train]

In [14]:
num_features, cat_features = get_num_cat_features(df_data, list(df_data.columns))

Vehicle_Youthful_Good_Student_Code
Vehicle_Driver_Points
Driver_Total_Female
Driver_Total_Male
Driver_Total_Teenager_Age_15_19
Driver_Total_College_Ages_20_23
Driver_Total_Young_Adult_Ages_24_29
Driver_Total_Low_Middle_Adult_Ages_30_39
Driver_Total_Middle_Adult_Ages_40_49
Driver_Total_Adult_Ages_50_64
Driver_Total_Senior_Ages_65_69
Driver_Total_Upper_Senior_Ages_70_plus
Driver_Total_Married
Driver_Total_Single
Vehicle_Usage
Vehicle_Miles_To_Work
Vehicle_Territory
Annual_Premium
Loss_Amount


In [15]:
num_features

['Vehicle_Driver_Points',
 'Driver_Total_Female',
 'Driver_Total_Male',
 'Driver_Total_Teenager_Age_15_19',
 'Driver_Total_College_Ages_20_23',
 'Driver_Total_Young_Adult_Ages_24_29',
 'Driver_Total_Low_Middle_Adult_Ages_30_39',
 'Driver_Total_Middle_Adult_Ages_40_49',
 'Driver_Total_Adult_Ages_50_64',
 'Driver_Total_Senior_Ages_65_69',
 'Driver_Total_Upper_Senior_Ages_70_plus',
 'Driver_Total_Married',
 'Driver_Total_Single',
 'Vehicle_Miles_To_Work',
 'Vehicle_Territory',
 'Annual_Premium',
 'Loss_Amount']

In [16]:
cat_features

['Vehicle_Youthful_Good_Student_Code', 'Vehicle_Usage']

In [17]:
df_num = df_data[num_features]
df_num.head()

,Vehicle_Driver_Points,Driver_Total_Female,Driver_Total_Male,Driver_Total_Teenager_Age_15_19,Driver_Total_College_Ages_20_23,Driver_Total_Young_Adult_Ages_24_29,Driver_Total_Low_Middle_Adult_Ages_30_39,Driver_Total_Middle_Adult_Ages_40_49,Driver_Total_Adult_Ages_50_64,Driver_Total_Senior_Ages_65_69,Driver_Total_Upper_Senior_Ages_70_plus,Driver_Total_Married,Driver_Total_Single,Vehicle_Miles_To_Work,Vehicle_Territory,Annual_Premium,Loss_Amount
0,0,1,0,0,0,0,0,0,0,0,1,0,0,-1,35,320.12,0.0
1,0,1,0,0,0,0,0,0,0,0,1,1,0,-1,35,259.70,0.0
2,0,0,1,0,0,0,0,0,0,0,1,1,0,-1,17,613.74,0.0
3,0,0,1,0,0,0,0,0,0,0,1,1,0,-1,17,541.66,0.0
4,0,0,1,0,0,0,0,0,0,0,1,1,0,-1,17,541.66,0.0


In [18]:
df_cat = df_data[cat_features]
df_cat.head()

,Vehicle_Youthful_Good_Student_Code,Vehicle_Usage
0,Not Eligible for Good Student Credit,Farm
1,Not Eligible for Good Student Credit,Farm
2,Not Eligible for Good Student Credit,Pleasure
3,Not Eligible for Good Student Credit,Pleasure
4,Not Eligible for Good Student Credit,Pleasure


In [19]:
df_cat_strip = strip_white_spaces(df_cat)

C:\Users\vishn\AppData\Local\Temp\ipykernel_29528\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()


In [20]:
df_data_clean = df_num.join(df_cat, how='outer')

In [21]:
df_data_clean.head()

,Vehicle_Driver_Points,Driver_Total_Female,Driver_Total_Male,Driver_Total_Teenager_Age_15_19,Driver_Total_College_Ages_20_23,Driver_Total_Young_Adult_Ages_24_29,Driver_Total_Low_Middle_Adult_Ages_30_39,Driver_Total_Middle_Adult_Ages_40_49,Driver_Total_Adult_Ages_50_64,Driver_Total_Senior_Ages_65_69,Driver_Total_Upper_Senior_Ages_70_plus,Driver_Total_Married,Driver_Total_Single,Vehicle_Miles_To_Work,Vehicle_Territory,Annual_Premium,Loss_Amount,Vehicle_Youthful_Good_Student_Code,Vehicle_Usage
0,0,1,0,0,0,0,0,0,0,0,1,0,0,-1,35,320.12,0.0,Not Eligible for Good Student Credit,Farm
1,0,1,0,0,0,0,0,0,0,0,1,1,0,-1,35,259.70,0.0,Not Eligible for Good Student Credit,Farm
2,0,0,1,0,0,0,0,0,0,0,1,1,0,-1,17,613.74,0.0,Not Eligible for Good Student Credit,Pleasure
3,0,0,1,0,0,0,0,0,0,0,1,1,0,-1,17,541.66,0.0,Not Eligible for Good Student Credit,Pleasure
4,0,0,1,0,0,0,0,0,0,0,1,1,0,-1,17,541.66,0.0,Not Eligible for Good Student Credit,Pleasure


In [22]:
def get_unique_values_with_count(df_input):
    columns = df_input.columns
    for column in columns:
        unique_value_count = df_input[column].value_counts()
        print(column)
        print()
        print(unique_value_count)
        print('--'*50)

In [23]:
get_unique_values_with_count(df_data)

Vehicle_Youthful_Good_Student_Code

Not Eligible for Good Student Credit    418562
Eligible for Good Student Credit          5869
Name: Vehicle_Youthful_Good_Student_Code, dtype: int64
----------------------------------------------------------------------------------------------------
Vehicle_Driver_Points

0    381289
1     36431
2      5905
3       670
4       108
5        21
6         5
7         2
Name: Vehicle_Driver_Points, dtype: int64
----------------------------------------------------------------------------------------------------
Driver_Total_Female

1    238676
0    181914
2      3837
3         4
Name: Driver_Total_Female, dtype: int64
----------------------------------------------------------------------------------------------------
Driver_Total_Male

1    222967
0    198925
2      2537
3         2
Name: Driver_Total_Male, dtype: int64
----------------------------------------------------------------------------------------------------
Driver_Total_Teenager_Age_15_19

0  

Features to be used in portfolio
Vehicle_Make_Year_Mean
Vehicle_Performance_Standard_Minus_all
Vehicle_Number_Of_Drivers_Assigned_Mean_Ignore_gt_10
Vehicle_Usage_Pleasure_Minus_all
Vehicle_Anti_Theft_Device_anything_but_not_applicable
Vehicle_Age_In_Years_Mean
Vehicle_Collision_Coverage_Indicator_Y_minus_N
Driver_Total_Single_Sum_not_include_0
Driver_Total_Mean
Driver_Total_Male_Mean
Driver_Total_Female_Mean
Driver_Total_Married_Mean
Driver_Total_Teenager_Age_15_19_Mean
Driver_Total_College_Ages_20_23_Sum
Driver_Total_Young_Adult_Ages_24_29_Mean
Driver_Total_Upper_Senior_Ages_70_plus_Mean
Annual_Premium_Sum

In [24]:
selected_features_test = ['Vehicle_Make_Year',
                        'Vehicle_Number_Of_Drivers_Assigned',
                        'Vehicle_Age_In_Years',
                        'Driver_Total_Single',
                        'Driver_Total',
                        'Driver_Total_Male',
                        'Driver_Total_Female',
                        'Driver_Total_Married',
                        'Driver_Total_Teenager_Age_15_19',
                        'Driver_Total_College_Ages_20_23',
                        'Driver_Total_Young_Adult_Ages_24_29',
                        'Driver_Total_Upper_Senior_Ages_70_plus',
                        'Vehicle_Usage',
                        'Vehicle_Anti_Theft_Device',
                        'Vehicle_Performance',
                        'Annual_Premium']

In [25]:
# Create Portfolios in training data like testing portfolios

In [26]:
df_data.shape

(424431, 19)

In [27]:
policies_in_portfolios = [5000, 10000, 15000]
percentage_losses = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20]

In [28]:
df_list_portfolios = []

In [29]:
df_data_oh = pd.get_dummies(df_data)

In [30]:
df_data_oh.shape

(424431, 23)

In [31]:
df_data_oh_columns = list(df_data_oh.columns)

In [32]:
for i in range(10):
    df_data = df_data_oh.sample(frac=1)
    df_data_loss_zero = df_data[df_data['Loss_Amount'] == 0]
    df_data_loss_non_zero = df_data[df_data['Loss_Amount'] != 0]
    current_df_without_loss = 0
    current_df_with_loss = 0
    for i in range(len(policies_in_portfolios)):
        for j in range(len(percentage_losses)):
            without_loss_size = int(((100 - percentage_losses[j])/100) * policies_in_portfolios[i])
            with_loss_size = int(policies_in_portfolios[i] - without_loss_size )

            df_without_loss = df_data_loss_zero.iloc[current_df_without_loss: current_df_without_loss + without_loss_size]
            df_with_loss = df_data_loss_non_zero.iloc[current_df_with_loss: current_df_with_loss + with_loss_size]

            current_df_without_loss = current_df_without_loss + without_loss_size
            current_df_with_loss = current_df_with_loss + with_loss_size

            df_merge = pd.concat([df_without_loss, df_with_loss])
            df_list_portfolios.append(df_merge)

In [33]:
len(df_list_portfolios)

330

In [34]:
df_list_portfolios[0].shape

(5000, 23)

In [35]:
df_list_portfolios[0].shape

(5000, 23)

In [36]:
df_data_final = None

In [37]:

df_data_oh_columns

['Vehicle_Driver_Points',
 'Driver_Total_Female',
 'Driver_Total_Male',
 'Driver_Total_Teenager_Age_15_19',
 'Driver_Total_College_Ages_20_23',
 'Driver_Total_Young_Adult_Ages_24_29',
 'Driver_Total_Low_Middle_Adult_Ages_30_39',
 'Driver_Total_Middle_Adult_Ages_40_49',
 'Driver_Total_Adult_Ages_50_64',
 'Driver_Total_Senior_Ages_65_69',
 'Driver_Total_Upper_Senior_Ages_70_plus',
 'Driver_Total_Married',
 'Driver_Total_Single',
 'Vehicle_Miles_To_Work',
 'Vehicle_Territory',
 'Annual_Premium',
 'Loss_Amount',
 'Vehicle_Youthful_Good_Student_Code_Eligible for Good Student Credit    ',
 'Vehicle_Youthful_Good_Student_Code_Not Eligible for Good Student Credit',
 'Vehicle_Usage_Business',
 'Vehicle_Usage_Farm    ',
 'Vehicle_Usage_Pleasure',
 'Vehicle_Usage_Work    ']

In [38]:
selected_features_mean = num_features.copy()
selected_features_mean.remove('Annual_Premium')
selected_features_mean.remove('Loss_Amount')
mean_columns = selected_features_mean

sum_columns = ['Annual_Premium']

for cat_f in cat_features:
    for col in list(df_list_portfolios[0].columns):
        if col.startswith(cat_f):
            sum_columns.append(col)

target_column = ['Loss_Amount']

In [39]:
sum_columns

['Annual_Premium',
 'Vehicle_Youthful_Good_Student_Code_Eligible for Good Student Credit    ',
 'Vehicle_Youthful_Good_Student_Code_Not Eligible for Good Student Credit',
 'Vehicle_Usage_Business',
 'Vehicle_Usage_Farm    ',
 'Vehicle_Usage_Pleasure',
 'Vehicle_Usage_Work    ']

In [40]:
ls_final_training = []

In [41]:
def portfolio_to_features(df_portfolio, is_training=True):
    df_portfolio_mean = df_portfolio[mean_columns]
    df_portfolio_mean = df_portfolio_mean.mean()
    ls_portfolio_mean = df_portfolio_mean.to_list()
    
    df_portfolio_sum = df_portfolio[sum_columns]
    df_portfolio_sum = df_portfolio_sum.sum()
    ls_portfolio_sum = df_portfolio_sum.to_list()
    
    if is_training:
        df_portfolio_target = df_portfolio[target_column]
        df_portfolio_target_sum = df_portfolio_target.sum()
        ls_portfolio_target_sum = df_portfolio_target_sum.to_list()
    
    list_merge = ls_portfolio_mean + ls_portfolio_sum
    
    if is_training:
        list_merge = list_merge + ls_portfolio_target_sum
    
    return list_merge

In [42]:
def features_to_training_df(df_portfolios_list):
    features_list = mean_columns + sum_columns + target_column
    final_list = []
    for single_portfolio in df_portfolios_list:
        ls_portfolio = portfolio_to_features(single_portfolio)
        final_list.append(ls_portfolio)
    df_final_training = pd.DataFrame(final_list, columns = features_list)
    return df_final_training

In [43]:
df_final_train = features_to_training_df(df_list_portfolios)

In [44]:

df_final_train.head()

,Vehicle_Driver_Points,Driver_Total_Female,Driver_Total_Male,Driver_Total_Teenager_Age_15_19,Driver_Total_College_Ages_20_23,Driver_Total_Young_Adult_Ages_24_29,Driver_Total_Low_Middle_Adult_Ages_30_39,Driver_Total_Middle_Adult_Ages_40_49,Driver_Total_Adult_Ages_50_64,Driver_Total_Senior_Ages_65_69,...,Vehicle_Miles_To_Work,Vehicle_Territory,Annual_Premium,Vehicle_Youthful_Good_Student_Code_Eligible for Good Student Credit,Vehicle_Youthful_Good_Student_Code_Not Eligible for Good Student Credit,Vehicle_Usage_Business,Vehicle_Usage_Farm,Vehicle_Usage_Pleasure,Vehicle_Usage_Work,Loss_Amount
0,0.1140,0.5836,0.5322,0.0394,0.0412,0.0924,0.2082,0.2642,0.3088,0.0596,...,3.3738,31.1222,1660733.14,51.0,4949.0,39.0,460.0,2690.0,1811.0,259884.90
1,0.1278,0.5762,0.5354,0.0418,0.0426,0.0900,0.2178,0.2536,0.3168,0.0582,...,3.2218,31.0234,1677052.09,63.0,4937.0,36.0,433.0,2743.0,1788.0,756196.84
2,0.1168,0.5802,0.5400,0.0434,0.0478,0.0814,0.2064,0.2742,0.3044,0.0624,...,3.2404,31.0686,1657744.62,74.0,4926.0,19.0,459.0,2678.0,1844.0,746759.49
3,0.1252,0.5816,0.5316,0.0444,0.0532,0.0916,0.2128,0.2488,0.3072,0.0606,...,3.3356,31.1156,1702077.81,70.0,4930.0,37.0,428.0,2673.0,1862.0,881115.85
4,0.1260,0.5832,0.5388,0.0432,0.0474,0.0902,0.2288,0.2586,0.2988,0.0606,...,3.1882,31.1026,1682616.24,69.0,4931.0,34.0,437.0,2733.0,1796.0,1177212.39


# Training the model with Decision Tree

In [45]:
X_train = df_final_train.loc[:, df_final_train.columns != 'Loss_Amount']
y_train = df_final_train['Loss_Amount']

X_train.shape, y_train.shape

((330, 22), (330,))

In [46]:
X_train.columns

Index(['Vehicle_Driver_Points', 'Driver_Total_Female', 'Driver_Total_Male',
       'Driver_Total_Teenager_Age_15_19', 'Driver_Total_College_Ages_20_23',
       'Driver_Total_Young_Adult_Ages_24_29',
       'Driver_Total_Low_Middle_Adult_Ages_30_39',
       'Driver_Total_Middle_Adult_Ages_40_49', 'Driver_Total_Adult_Ages_50_64',
       'Driver_Total_Senior_Ages_65_69',
       'Driver_Total_Upper_Senior_Ages_70_plus', 'Driver_Total_Married',
       'Driver_Total_Single', 'Vehicle_Miles_To_Work', 'Vehicle_Territory',
       'Annual_Premium',
       'Vehicle_Youthful_Good_Student_Code_Eligible for Good Student Credit    ',
       'Vehicle_Youthful_Good_Student_Code_Not Eligible for Good Student Credit',
       'Vehicle_Usage_Business', 'Vehicle_Usage_Farm    ',
       'Vehicle_Usage_Pleasure', 'Vehicle_Usage_Work    '],
      dtype='object')

In [ ]:
# Decision Tree Regressor

In [54]:
decision_tree_regressor = DecisionTreeRegressor()
decision_tree_regressor.fit(X_train, y_train)
y_train_pred = decision_tree_regressor.predict(X_train)
mae = mean_absolute_error(y_train, y_train_pred)
mae

0.0

In [1]:
# XGBRegressor

In [48]:
from xgboost import XGBRegressor
xg_regressor = XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.01, max_depth = 3, alpha = 10, n_estimators = 100)
xg_regressor.fit(np.nan_to_num(X_train.to_numpy()), y_train)

y_train_pred_xg = xg_regressor.predict(np.nan_to_num(X_train.to_numpy()))

mae_xg = mean_absolute_error(y_train, y_train_pred_xg)
mae_xg

1400307.0250757579

In [53]:
from xgboost import XGBRegressor
xg_regressor = XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.01, max_depth = 3, alpha = 10, n_estimators = 100)
xg_regressor.fit(X_train,y_train)

y_train_pred_xg = xg_regressor.predict(X_train)

mae_xg = mean_absolute_error(y_train, y_train_pred_xg)
mae_xg


1384504.9320833334

In [ ]:
#Linear Regressor

In [49]:
from sklearn.linear_model import LinearRegression

# Create linear regression object
regressor = LinearRegression()

# Fit model to training data
regressor.fit(X_train,y_train)

y_pred = regressor.predict(X_train)

# Score It
from sklearn import metrics
print('MAE:',metrics.mean_absolute_error(y_train,y_pred))
print('MSE:',metrics.mean_squared_error(y_train,y_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_train,y_pred)))

rmse_linear = np.sqrt(metrics.mean_squared_error(y_train,y_pred))

# Calculated R Squared
print('R^2 =',metrics.explained_variance_score(y_train,y_pred))

MAE: 668642.0937472999
MSE: 761589019744.4069
RMSE: 872690.6781583077
R^2 = 0.8592808273832641


In [50]:
# Using RandomForestRegressor to evaluate model

from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics


regressor = RandomForestRegressor(n_estimators = 50, random_state = 0);
# training the random forest regressor model.
regressor.fit(X_train, y_train.values.ravel());
# Predicting the Test set results
rftree_pred = regressor.predict(X_train);

print('Mean Absolute Error:', metrics.mean_absolute_error(y_train, rftree_pred))  ;
print('Mean Squared Error:', metrics.mean_squared_error(y_train, rftree_pred)) ; 
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_train, rftree_pred)));
print('R^2 =',metrics.explained_variance_score(y_train,rftree_pred))

rmse_random = np.sqrt(metrics.mean_squared_error(y_train, rftree_pred))

Mean Absolute Error: 331869.8483121213
Mean Squared Error: 190878742075.2658
Root Mean Squared Error: 436896.7178581979
R^2 = 0.9647335018961644


In [ ]:
# KNN

In [51]:
knn_regressor = KNeighborsRegressor(5)
knn_regressor.fit(np.nan_to_num(X_train.to_numpy()), y_train)
y_train_pred_knn = knn_regressor.predict(np.nan_to_num(X_train.to_numpy()))

mae_knn = mean_absolute_error(y_train, y_train_pred_knn)
mae_knn

521705.9911878788

In [52]:
def load_test_portfolios():
    path_testing_dataset = "testing_portfolios"
    portfolio_files = os.listdir(path_testing_dataset)
    portfolio_dict = {}
    for i in range(len(portfolio_files)):
        portfolio_id = int(portfolio_files[i].split('.')[0][15:])
        portfolio_dict[portfolio_id] = portfolio_files[i]
    od = collections.OrderedDict(sorted(portfolio_dict.items()))
    po_list = []
    for key,val in od.items():
        po_list.append(val)
    df_portfolio_list = []
    for portfolio_file in po_list:
        file_path = os.path.join(path_testing_dataset, portfolio_file)
        df_test_portfolio = pd.read_csv(file_path)
        test_num_features = num_features.copy()
        test_num_features.remove('Loss_Amount')
        
        df_test_num = df_test_portfolio[test_num_features]
        df_test_cat = df_test_portfolio[cat_features]
        df_test_cat_strip = strip_white_spaces(df_test_cat)
        df_test_portfolio = df_test_num.join(df_test_cat_strip, how='outer')
        df_test_portfolio = pd.get_dummies(df_test_portfolio)
        df_test_portfolio = df_test_portfolio.reindex(sorted(X_train.columns), axis=1)
        
        df_portfolio_list.append(df_test_portfolio)
        
    result_dict = collections.OrderedDict()
    for i in range(len(po_list)):
        portfolio_id = po_list[i].split('.')[0][5:]
        result_dict[portfolio_id] = df_portfolio_list[i] 
    return result_dict

In [53]:
def features_to_testing_df(df_portfolios_ordered_dict):
    features_list = mean_columns + sum_columns
    features_list.insert(0, 'ID')
    final_list = []
    for portfolio_id, single_portfolio in df_portfolios_ordered_dict.items():
        ls_portfolio = portfolio_to_features(single_portfolio, False)
        ls_portfolio.insert(0, portfolio_id)
        final_list.append(ls_portfolio)
    df_final_testing = pd.DataFrame(final_list, columns = features_list)
    return df_final_testing

In [54]:
test_portfolios_dict = load_test_portfolios()

C:\Users\vishn\AppData\Local\Temp\ipykernel_30432\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\vishn\AppData\Local\Temp\ipykernel_30432\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\vishn\AppData\Local\Temp\ipykernel_30432\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

C:\Users\vishn\AppData\Local\Temp\ipykernel_30432\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\vishn\AppData\Local\Temp\ipykernel_30432\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\vishn\AppData\Local\Temp\ipykernel_30432\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

C:\Users\vishn\AppData\Local\Temp\ipykernel_30432\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\vishn\AppData\Local\Temp\ipykernel_30432\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\vishn\AppData\Local\Temp\ipykernel_30432\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

C:\Users\vishn\AppData\Local\Temp\ipykernel_30432\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\vishn\AppData\Local\Temp\ipykernel_30432\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\vishn\AppData\Local\Temp\ipykernel_30432\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

C:\Users\vishn\AppData\Local\Temp\ipykernel_30432\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\vishn\AppData\Local\Temp\ipykernel_30432\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\vishn\AppData\Local\Temp\ipykernel_30432\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

...

C:\Users\vishn\AppData\Local\Temp\ipykernel_30432\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\vishn\AppData\Local\Temp\ipykernel_30432\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\vishn\AppData\Local\Temp\ipykernel_30432\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

C:\Users\vishn\AppData\Local\Temp\ipykernel_30432\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\vishn\AppData\Local\Temp\ipykernel_30432\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\vishn\AppData\Local\Temp\ipykernel_30432\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

C:\Users\vishn\AppData\Local\Temp\ipykernel_30432\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\vishn\AppData\Local\Temp\ipykernel_30432\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\vishn\AppData\Local\Temp\ipykernel_30432\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

C:\Users\vishn\AppData\Local\Temp\ipykernel_30432\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\vishn\AppData\Local\Temp\ipykernel_30432\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\vishn\AppData\Local\Temp\ipykernel_30432\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

C:\Users\vishn\AppData\Local\Temp\ipykernel_30432\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\vishn\AppData\Local\Temp\ipykernel_30432\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[column] = input_df[column].str.strip()
C:\Users\vishn\AppData\Local\Temp\ipykernel_30432\2427808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [55]:

for key, value in test_portfolios_dict.items():
    print(key, value.shape)

portfolio_1 (1000, 22)
portfolio_2 (999, 22)
portfolio_3 (1000, 22)
portfolio_4 (997, 22)
portfolio_5 (998, 22)
portfolio_6 (1000, 22)
portfolio_7 (1000, 22)
portfolio_8 (998, 22)
portfolio_9 (998, 22)
portfolio_10 (998, 22)
portfolio_11 (1000, 22)
portfolio_12 (999, 22)
portfolio_13 (999, 22)
portfolio_14 (997, 22)
portfolio_15 (1000, 22)
portfolio_16 (999, 22)
portfolio_17 (998, 22)
portfolio_18 (999, 22)
portfolio_19 (998, 22)
portfolio_20 (997, 22)
portfolio_21 (1000, 22)
portfolio_22 (999, 22)
portfolio_23 (1000, 22)
portfolio_24 (1000, 22)
portfolio_25 (998, 22)
portfolio_26 (1000, 22)
portfolio_27 (998, 22)
portfolio_28 (999, 22)
portfolio_29 (998, 22)
portfolio_30 (998, 22)
portfolio_41 (1000, 22)
portfolio_42 (1000, 22)
portfolio_43 (998, 22)
portfolio_44 (999, 22)
portfolio_45 (999, 22)
portfolio_46 (999, 22)
portfolio_47 (998, 22)
portfolio_48 (1000, 22)
portfolio_49 (999, 22)
portfolio_50 (999, 22)
portfolio_51 (999, 22)
portfolio_52 (998, 22)
portfolio_53 (996, 22)
portfol

In [56]:
df_test_final = features_to_testing_df(test_portfolios_dict)

In [57]:
def generate_test_results(regressor, X_test, output_dir):
    # for each policy
    result_data = collections.OrderedDict()
    portfolio_ids_df = X_test.iloc[:, 0:1]
    X_test_features = X_test.iloc[:, 1:]
    
    y_pred = regressor.predict(np.nan_to_num(X_test_features.to_numpy()))
    loss_ratio = y_pred/X_test['Annual_Premium']
    loss_ratio_log = np.log(loss_ratio)
    loss_ratio_log_df = loss_ratio_log.to_frame(name='ln_LR')
    result_df = portfolio_ids_df.join(loss_ratio_log_df, how='outer')
    
    result_df.to_csv('output.csv', index=False)

In [58]:
generate_test_results(xg_regressor, df_test_final, '')